# Семинар 14

## Векторизация

**Векторизация текста**, то есть превращение текста в **векторы** (наборы чисел), — один из важнейших механизмов в NLP, который позволяет решать множество повседневных задач. Рассмотрим это явление на проблеме *выделения ключевых слов* в тексте. Мы уже пытались делать частотный словарь, но, как вы помните, самыми частотными оказывались служебные («в», «и», «это») или частотные лексические слова с абстрактным значением («сказать», «всегда»). А что если мы хотим выделить слова, которые *характеризуют* текст, сообщают что-то о его тематике? Попробуем разобраться, как это сделать.

### Введение

Для простоты рассмотрим этот вопрос на игрушечном корпусе текстов. Каждый из элементов списка ниже — отдельный текст. Например, первый текст — `"арбуз"`, а последний — `"банан и банан и банан и ещё один банан"`.

In [1]:
play_corpus = [
    "арбуз",
    "помидор и томат",
    "чай и печеньки",
    "сок и печеньки",
    "банан и банан и банан и ещё один банан",
]

Попробуем вручную (пока без кода) посчитать важность слов для конкретных текстов различными методами, постепенно усложняя вычисления.

_____

Первый вариант — уже знакомый вам «частотный словарь». На самом деле не совсем корректно это называть частотным словарём, потому что мы пока что считаем не *частоту*, а просто *количество* употреблений слова в тексте. По-умному такой метод называется «**мешок слов**», или _Bag of Words_.
- например, для текста «банан и банан и банан и ещё один банан»:
  - $4$ употребления _банан_ в этом тексте
  - $3$ употребление _и_ в этом тексте
  - $1$ употребление _ещё_ в этом тексте
  - $…$

Визуализируем это в виде таблицы, где столбцы — слова, строки — тексты, а в ячейках записано число употреблений слова в тексте.

|                                            | **арбуз** | **банан** | **ещё** | **и** | **один** | **печеньки** | **помидор** | **сок** | **томат** | **чай** |
|--------------------------------------------|-----------|-----------|---------|-------|----------|--------------|-------------|---------|-----------|----------|
| **арбуз**                                  | 1         | 0         | 0       | 0     | 0        | 0            | 0           | 0       | 0         | 0        |
| **помидор и томат**                        | 0         | 0         | 0       | 1     | 0        | 0            | 1           | 0       | 1         | 0        |
| **чай и печеньки**                         | 0         | 0         | 0       | 1     | 0        | 1            | 0           | 0       | 0         | 1        |
| **сок и печеньки**                         | 0         | 0         | 0       | 1     | 0        | 1            | 0           | 1       | 0         | 0        |
| **банан и банан и банан и ещё один банан** | 0         | 4         | 1       | 3     | 1        | 0            | 0           | 0       | 0         | 0        |


_____

Шаг второй. Попробуем учесть тот факт, что тексты могут быть разной длины. Для этого сделаем **частотный словарь**, то есть теперь уже реально посчитаем **частотность**, а не количество вхождений слова в тексте. Ещё такой процесс называется **нормализацией**, потому что после него все тексты будут иметь одинаковый вес в корпусе, несмотря на различия в размерах (сумма значений в каждом ряду будет равна $1$):
- абсолютные значения (кол-во употреблений слова в тексте) → относительные значения (процент употреблений слова в тексте относительно других слов в тексте)
- например, для текста «чай и печеньки»:
  - $1$ употребление _чай_ в этом тексте $/$ всего $3$ токенов в этом тексте $→ \frac{1}{3} ≈ 0,33$
  - $1$ употребление _и_ в этом тексте $/$ всего $3$ токенов в этом тексте $→ \frac{1}{3} ≈ 0,33$
  - $1$ употребление _печеньки_ в этом тексте $/$ всего $3$ токенов в этом тексте $→ \frac{1}{3} ≈ 0,33$

|                                            | **арбуз** | **банан** | **ещё** | **и**  | **один** | **печеньки** | **помидор** | **сок** | **томат** | **чай** |
|--------------------------------------------|-----------|-----------|---------|--------|----------|--------------|-------------|---------|-----------|----------|
| **арбуз**                                  | 1         | 0         | 0       | 0      | 0        | 0            | 0           | 0       | 0         | 0        |
| **помидор и томат**                        | 0         | 0         | 0       | 0.3333 | 0        | 0            | 0.3333      | 0       | 0.3333    | 0        |
| **чай и печеньки**                         | 0         | 0         | 0       | 0.3333 | 0        | 0.3333       | 0           | 0       | 0         | 0.3333   |
| **сок и печеньки**                         | 0         | 0         | 0       | 0.3333 | 0        | 0.3333       | 0           | 0.3333  | 0         | 0        |
| **банан и банан и банан и ещё один банан** | 0         | 0.4444    | 0.1111  | 0.3333 | 0.1111   | 0            | 0           | 0       | 0         | 0        |


_____

Шаг третий. Надо также учесть, что некоторые слова часто встречаются во всех текстах, а другие — только в некоторых (именно они-то нам и нужны). Для этого нам нужно посчитать **частотность употреблений каждого слова во всех текстах**. Если слово употребительно во всех текстах, о каждом конкретном тексте оно ничего не сообщает — а вот если оно часто появляется только в одном или нескольких текстах, вероятно, оно отражает их уникальные особенности. (Ниже мы рассмотрим, как это сделать в питоне с помощью особого инструмента `TfidfVectorizer`.)

- **стандартный вариант** (называется **TF-IDF**): разделить частотность слова в тексте на *число всех текстов*, в которых это слово встречается (_document frequency_)
- **альтернативный вариант**: разделить частотность слова в тексте на *число вхождений этого слова во всём корпусе*
- например, реализуем стандартный вариант для текста «чай и печеньки»:
  - $0,33$ $/$ всего $1$ текст в корпусе, в котором встречается слово _чай_ $→ 0,33$
  - $0,33$ $/$ всего $4$ текста в корпусе, в котором встречается слово _и_ $→ \frac{(1/3)}{4} = \frac{1}{12} ≈ 0,09$
  - $0,33$ $/$ всего $2$ текста в корпусе, в котором встречается слово _печеньки_ $→ \frac{(1/3)}{2} = \frac{1}{6} ≈ 0,16$
- таким образом, слово _чай_ оказывается важнее (хорошо характеризует этот текст), слово _печеньки_ чуть менее важно (хуже характеризирует этот текст), и слово _и_ совсем неважно (бессмысленно для характеристики текста)

> В **`TfidfVectorizer`** используется немного математически улучшенная версия этой формулы. Числа, которые она выдаёт, слегка отличаются от приведённых выше, но главное, что слова ранжируются похожим образом: в тексте «чай и печеньки» наиболее важным оказывается слово _чай_ ($0,42$), чуть менее важным — _печеньки_ ($0,34$), совсем маловажным — союз _и_ ($0,24$):

|                                            | **арбуз** | **банан** | **ещё** | **и**  | **один** | **печеньки** | **помидор** | **сок** | **томат** | **чай** |
|--------------------------------------------|-----------|-----------|---------|--------|----------|--------------|-------------|---------|-----------|----------|
| **арбуз**                                  | 1         | 0         | 0       | 0      | 0        | 0            | 0           | 0       | 0         | 0        |
| **помидор и томат**                        | 0         | 0         | 0       | 0.2198 | 0        | 0            | 0.3901      | 0       | 0.3901    | 0        |
| **чай и печеньки**                         | 0         | 0         | 0       | 0.2377 | 0        | 0.3404       | 0           | 0       | 0         | 0.4219   |
| **сок и печеньки**                         | 0         | 0         | 0       | 0.2377 | 0        | 0.3404       | 0           | 0.4219  | 0         | 0        |
| **банан и банан и банан и ещё один банан** | 0         | 0.5201    | 0.13    | 0.2198 | 0.13     | 0            | 0           | 0       | 0         | 0        |

### TF-IDF

***TF-IDF*** — известная метрика, которая позволяет оценить относительную важность слова для каждого текста в корпусе. По сути мы уже рассмотрели её выше.

Рассчитывается она так:

TF (*term frequency*) — **частотность слова** в тексте
- $TF(w, t) =$ (кол-во токенов, т.е. употреблений слова $w$ в тексте $t$) $/$ (кол-во всех токенов в тексте $t$)

DF (*document frequency*) — **частотность текстов**, содержащих это слово, в корпусе
- $DF(w, T) =$ (кол-во текстов, в которых есть слово $w$) $/$ (кол-во текстов в корпусе $T$)
- в TF-IDF используется IDF (*inverse document frequency*) — «обратная» частотность
- рассчитывается она как $\frac{1}{DF}$, обычно с добавлением логарифма
- $→$ $IDF(w, T) = log(\frac{1}{DF(w, T)})$
  - логарифм «сглаживает» полученные числа и делает их более удобными для сравнения (это не меняет сути формулы, и понимать причину и математический смысл логарифма здесь необязательно)

Итоговая формула: $TFIDF(w, t) = TF(w, t) × IDF(w, T)$.

> Чем выше получившееся значение *TF-IDF*, тем более важно данное слово для конкретного текста.

### Реализация TF-IDF в модуле **`sklearn`**

TF-IDF можно написать с нуля, а можно использовать его реализацию в библиотеке `scikit-learn` (в питоне используется более короткое название **`sklearn`**). Это известная и широко используемая библиотека, содержащая кучу инструментов для машинного обучения. Импортируем класс `TfidfVectorizer` из подмодуля `sklearn.feature_extraction.text`, и заодно также импортируем стандартную токенизацию из `nltk`:

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.tokenize import word_tokenize

При создании объекта типа **`TfidfVectorizer`** можно не подавать ничего, а можно указать необходимые настройки (см. [подробнее здесь](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)). Рассмотрим некоторые из них:
- `tokenizer` — можно подать свою функцию для токенизации текста на токены (например, `nltk.word_tokenize`); программа будет сама токенизировать текст с помощью этой функции. Если не упоминать этот аргумент, будет просто разделение по пробелам
- `stop_words` — можно подать свой список стоп-слов. Если не упоминать этот аргумент, фильтрации по стоп-словам не будет
- `use_idf` — по умолчанию `True`. Если отключить (подав `False`), то IDF считаться не будет, и программа будет возвращать просто TF
- `norm` — три опции, `None`, `"l1"`, `"l2"`
  - если `None`, то возвращаются просто ненормализованные значения (частотный словарь с абсолютными количествами токенов)
  - если `"l1"`, то происходит обычная нормализация, которую мы обсуждали выше (сумма значений для каждого текста равна 1)
  - если `"l1"`, то происходит немного более сложная нормализация (сумма _квадратов_ значений для каждого текста равна 1). По некоторым математическим причинам это работает чуть лучше обычной нормализации, поэтому это значение `norm` по умолчанию

Создадим экземпляр `TfidfVectorizer`, чтобы проверить его работу на нашем игрушечном корпусе. Поставим для начала самые простые настройки: IDF не считать, нормализацию не использовать:

In [3]:
vectorizer = TfidfVectorizer(tokenizer=word_tokenize, use_idf=False, norm=None)

Мы получили объект-векторизатор. Применим метод `.fit_transform()` — он принимает на вход корпус текстов и создаёт матрицу с числом употреблений токенов в текстах. Эта матрица хранится в специальном «сжатом» формате. Чтобы посмотреть на конкретные числа, к ней нужно применить метод `.toarray()`. Сделаем так и увидим, что мы получили нужную матрицу, которую уже подсчитали вручную выше. (Предупреждение, которое выдаёт питон, в данном случае можно смело проигнорировать.)

In [4]:
vector_matrix = vectorizer.fit_transform(play_corpus)
vector_array = vector_matrix.toarray()
vector_array

C:\Users\Samsung\AppData\Roaming\Python\Python311\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 1., 0., 1., 0.],
       [0., 0., 0., 1., 0., 1., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 1., 0., 1., 0., 0.],
       [0., 4., 1., 3., 1., 0., 0., 0., 0., 0.]])

Такую матрицу можно сделать гораздо более наглядной с помощью библиотеки **`pandas`**, которая позволяет работать с данными в табличном формате; обычно её импортируют под псевдонимом `pd`. Из матрицы можно создать объект `pd.DataFrame`, то есть датафрейм, а попросту — таблицу, в которой будут видны названия столбцов и строк. (Изучение `pandas` не входит в наши планы, поэтому разбираться в коде ячейки ниже не нужно, это просто более наглядная демонстрация для понимания работы TF-IDF.)

In [5]:
import pandas as pd

corpus_vocabulary = vectorizer.get_feature_names_out()
vector_table = pd.DataFrame(vector_array,
                            columns=corpus_vocabulary,   # названия колонок — токены
                            index=play_corpus)           # названия строк («индексы») — названия текстов
vector_table

,арбуз,банан,ещё,и,один,печеньки,помидор,сок,томат,чай
арбуз,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
помидор и томат,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
чай и печеньки,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
сок и печеньки,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
банан и банан и банан и ещё один банан,0.0,4.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0


Теперь попробуем то же самое ещё раз, только используем дефолтные настройки `TfidfVectorizer`: пусть он нормализует данные и учитывает IDF. (Токенизацию всё же лучше оставить, потому что дефолтная токенизация, встроенная в `TfidfVectorizer`, зачем-то выкидывает из текста однобуквенные слова, а у нас тут союз «и» встречается.)

In [6]:
vectorizer = TfidfVectorizer(tokenizer=word_tokenize)

vector_matrix = vectorizer.fit_transform(play_corpus)

corpus_vocabulary = vectorizer.get_feature_names_out()
vector_table = pd.DataFrame(vector_matrix.toarray(),
                            columns=corpus_vocabulary,
                            index=play_corpus)
vector_table

C:\Users\Samsung\AppData\Roaming\Python\Python311\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,арбуз,банан,ещё,и,один,печеньки,помидор,сок,томат,чай
арбуз,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000
помидор и томат,0.0,0.000000,0.000000,0.370086,0.000000,0.000000,0.6569,0.000000,0.6569,0.000000
чай и печеньки,0.0,0.000000,0.000000,0.401565,0.000000,0.575063,0.0000,0.000000,0.0000,0.712775
сок и печеньки,0.0,0.000000,0.000000,0.401565,0.000000,0.575063,0.0000,0.712775,0.0000,0.000000
банан и банан и банан и ещё один банан,0.0,0.875867,0.218967,0.370086,0.218967,0.000000,0.0000,0.000000,0.0000,0.000000


До конкретных значений TF-IDF можно добраться так, как будто таблица представляет из себя словарь: сначала записать в виде «ключа» (в квадратных скобках) название столбца, а затем (в ещё одних скобках) название строки:

In [12]:
print("важность слова «и» по текстам")
print(vector_table["и"])

важность слова «и» по текстам
арбуз                                     0.000000
помидор и томат                           0.370086
чай и печеньки                            0.401565
сок и печеньки                            0.401565
банан и банан и банан и ещё один банан    0.370086
Name: и, dtype: float64


In [13]:
print("важность слова «и» в тексте «банан и банан и банан и ещё один банан»")
print(vector_table["и"]["банан и банан и банан и ещё один банан"])

важность слова «и» в тексте «банан и банан и банан и ещё один банан»
0.3700862108940938


In [14]:
for word in ("чай", "печеньки", "и", "арбуз"):
    print(f"важность слова «{word}» в тексте «чай и печеньки»",
          vector_table[word]["чай и печеньки"])

важность слова «чай» в тексте «чай и печеньки» 0.7127752157729959
важность слова «печеньки» в тексте «чай и печеньки» 0.5750625560879445
важность слова «и» в тексте «чай и печеньки» 0.4015651234424611
важность слова «арбуз» в тексте «чай и печеньки» 0.0
